In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageDraw
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [7]:
import pandas as pd
import plotly.graph_objects as go

def create_rankings(df, years):
    rank_columns = ["rank_{}".format(i) for i in range(len(years))]
    for i, year in enumerate(years):
        df[rank_columns[i]] = df.groupby('year')['new_rating'].rank(ascending=False)
        
    return df, rank_columns

def add_text(fig, beer_name, x, y, text, font_size=12):
    fig.add_trace(go.Scatter(x=[x], y=[y], text=[text], mode="text", showlegend=False,
                             textfont=dict(size=font_size), hoverinfo='text+x+y'))

def add_pie_and_flag(fig, x, y, name, ratio):
    fig.add_trace(go.Pie(labels=[name], values=[ratio], hole=0.4,
                         marker=dict(colors=['rgba(255, 255, 255, 0)']),
                         domain=dict(x=[x-0.15, x+0.15], y=[y-0.15, y+0.15]),
                         hoverinfo='label+percent+name', showlegend=False))

years = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"]
df = pd.read_csv("./../../Project/top_10_beers_by_year.csv", index_col=None)
df, rank_columns = create_rankings(df, years)


Unnamed: 0                           1
beer_name     Weihenstephaner Original
year                            2000.0
new_rating                    1.272458
rank_0                             1.0
rank_1                             1.0
rank_2                             1.0
rank_3                             1.0
rank_4                             1.0
rank_5                             1.0
rank_6                             1.0
rank_7                             1.0
rank_8                             1.0
rank_9                             1.0
rank_10                            1.0
rank_11                            1.0
rank_12                            1.0
rank_13                            1.0
rank_14                            1.0
rank_15                            1.0
rank_16                            1.0
rank_17                            1.0
Name: 0, dtype: object


KeyError: 1.0

In [127]:
import pandas as pd
from raceplotly.plots import barplot

data = pd.read_csv('./../ada-2023-project-laremontada61/top_10_beers_by_year.csv')
my_raceplot = barplot(data,
                      item_column='beer_name',
                      value_column='new_rating',
                      time_column='year')

my_raceplot.plot(title = 'Top 10 beers in the world from 2000 to 2017',
                 item_label = 'Top 10 beers',
                 value_label = 'New rating',
                 frame_duration = 800)